In [18]:
# Desabilitando warnings
import warnings
warnings.filterwarnings("ignore")

# Datathon - Sistema de recomendação de noticias

# Download Data

prmeiramente estou fazendo o download do arquivo. Estou usando o stream e salvando o arquivo incementalmente, pois o arquivo é relativamente grande e possui mais de 1 GB.

In [5]:
import requests

URL = r'https://drive.usercontent.google.com/download?id=13rvnyK5PJADJQgYe-VbdXb7PpLPj7lPr&export=download&authuser=0&confirm=t&uuid=02f630ab-0d33-489e-ae09-45facc58edfa&at=APvzH3qdyuGKBM5eXLv9nq2BIPb3%3A1736166348726'

response = requests.get(URL, stream=True)

with open('data.zip', 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

Extraindo o conteudo do arquivo para uma pasta de dados para que possamos analizar o seu conteúdo.

In [7]:
import zipfile
import os

os.makedirs('data', exist_ok=True)

with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

os.remove('data.zip')

os arquivos foram baixados, podemos começar a verificar os dados baixados.

# Conferindo os dados

In [8]:
import pandas as pd

In [17]:
arquivos = [ f'./data/files/treino/treino_parte{n}.csv' for n in range(1, 7) ]
dfs = [ pd.read_csv(arquivo) for arquivo in arquivos ]
df_treino = pd.concat(dfs)

del arquivos, dfs

df_treino = df_treino.drop(columns=['timestampHistory_new'])

df_treino.head(1)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1"


Esses são os dados de acesso às matérias do G1, nesse dataframe temos os dados de atividade dos usuários da plataforma de notícias, além de tempo de leitura, percentual de rolagem da pagina, entre outros.

Cada linha apresenta os dados de 1 unico usuário, tendo as informações de acesso em um formato que dificulta a utilização. Alguns ajustes na formatação do arquivo podem facilitar o processo de análise dos dados.

In [18]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577942 entries, 0 to 77941
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   userId                   577942 non-null  object
 1   userType                 577942 non-null  object
 2   historySize              577942 non-null  int64 
 3   history                  577942 non-null  object
 4   timestampHistory         577942 non-null  object
 5   numberOfClicksHistory    577942 non-null  object
 6   timeOnPageHistory        577942 non-null  object
 7   scrollPercentageHistory  577942 non-null  object
 8   pageVisitsCountHistory   577942 non-null  object
dtypes: int64(1), object(8)
memory usage: 44.1+ MB


In [10]:
arquivos = [ f'./data/itens/itens/itens-parte{n}.csv' for n in range(1, 4) ]
dfs = [ pd.read_csv(arquivo) for arquivo in arquivos ]
df_itens = pd.concat(dfs)

del arquivos, dfs

df_itens.head(1)

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...


Esse arquivo contém as informações das matérias disponíveis no portal G1. Essas informações podem ser relevantes para buscar as correlações entre as matérias e enriquecer nosso sistema de recomendação.

# Análise e normalização de dados

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

## Normalização dos dados

Primeiramente quero transformar os dados tanto em formato em que estão estruturados, quanto em relação à tipagem. Essas transformações devem facilitar os trabalhos futuros de manipulação e visualização desses dados.

In [2]:
if not os.path.exists('data/files/treino/long_format.csv'):    
    colunas_separadas_por_virgula = ['history', 'timestampHistory', 'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory']

    for coluna in colunas_separadas_por_virgula:
        df_treino[coluna] = df_treino[coluna].str.split(',')

    df_treino = df_treino.explode(colunas_separadas_por_virgula)

    df_treino.to_csv(r'data/files/treino/long_format.csv', index=False)

else:
    df_treino = pd.read_csv(r'data/files/treino/long_format.csv')

In [3]:
df_treino.head(3)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,c8aab885-433d-4e46-8066-479f40ba7fb2,1657146417045,76,20380,50.30,2
1,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,68d2039c-c9aa-456c-ac33-9b2e8677fba7,1657146605778,38,21184,18.18,1
2,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,13e423ce-1d69-4c78-bc18-e8c8f7271964,1657146698738,41,35438,16.46,1


In [4]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8123951 entries, 0 to 8123950
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   userId                   object 
 1   userType                 object 
 2   historySize              int64  
 3   history                  object 
 4   timestampHistory         int64  
 5   numberOfClicksHistory    int64  
 6   timeOnPageHistory        int64  
 7   scrollPercentageHistory  float64
 8   pageVisitsCountHistory   int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 557.8+ MB


In [5]:
df_treino['userType'] = df_treino['userType'].astype('category')
df_treino['historySize'] = df_treino['historySize'].astype('int')
df_treino['timestampHistory'] = pd.to_datetime(df_treino['timestampHistory'], unit='ms')
df_treino['numberOfClicksHistory'] = df_treino['numberOfClicksHistory'].astype('int')
df_treino['timeOnPageHistory'] = df_treino['timeOnPageHistory'].astype('int')
df_treino['scrollPercentageHistory'] = df_treino['scrollPercentageHistory'].astype('float')
df_treino['pageVisitsCountHistory'] = df_treino['pageVisitsCountHistory'].astype('int')

In [6]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8123951 entries, 0 to 8123950
Data columns (total 9 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   userId                   object        
 1   userType                 category      
 2   historySize              int64         
 3   history                  object        
 4   timestampHistory         datetime64[ns]
 5   numberOfClicksHistory    int64         
 6   timeOnPageHistory        int64         
 7   scrollPercentageHistory  float64       
 8   pageVisitsCountHistory   int64         
dtypes: category(1), datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 503.6+ MB


In [7]:
df_treino.head(1)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,c8aab885-433d-4e46-8066-479f40ba7fb2,2022-07-06 22:26:57.045,76,20380,50.3,2


In [2]:
if not os.path.exists(r'data/itens/itens/long_format.csv'):
    df_itens.to_csv(r'data/itens/itens/long_format.csv', index=False)

else:
    df_itens = pd.read_csv(r'data/itens/itens/long_format.csv')

In [3]:
df_itens.head(1)

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...


## Análise de dados

### Analise de itens

In [4]:
print(df_itens.iloc[0]['title'])
print('-----------------------------------')
print(df_itens.iloc[0]['caption'])
print('-----------------------------------')
print(df_itens.iloc[0]['body'])

Caso Bruno e Dom: 3º suspeito tem prisão temporária decretada pela Justiça do AM
-----------------------------------
Jeferson da Silva Lima foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia
-----------------------------------
Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábado (18), a prisão temporária,  por 30 dias, de Jeferson da Silva Lima, conhecido como "Pelado da Dinha". Ele teve participação direta na morte do indigenista Bruno Pereira e do jornalista inglês Dom Phillips, aponta as investigações. 
"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido e não ser localizado pelas autoridades. Ele se entregou na delegacia de Atalaia do Norte, a 1.136 quilômetros de Manaus, nas primeiras horas da manhã deste sábado, onde foi ouvido pelo delegado Alex Perez Timóteo. 
"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após t

primeiramente quero testar a possibilidade de utilizar tags para simplificar a busca de matérias com conteúdos similates.

In [5]:
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
pt_stopwords = list(stopwords.words('portuguese'))

def teste_tags(feature:str, top_n=25):
    vectorizer = TfidfVectorizer(stop_words=pt_stopwords)
    X = vectorizer.fit_transform(df_itens[feature])

    termos = vectorizer.get_feature_names_out()

    def obter_tags(indice_texto, top_n=top_n):
        tfidf_scores = X[indice_texto].toarray().flatten()
        indices = tfidf_scores.argsort()[-top_n:][::-1]
        return [termos[i] for i in indices]
        

    for i, texto in enumerate(df_itens[feature][:5]):
        print(f"Texto: {texto[:80]}")
        print(f"Tags: {' '.join(obter_tags(i))}\n")

In [6]:
' '.join(pt_stopwords)

'a à ao aos aquela aquelas aquele aqueles aquilo as às até com como da das de dela delas dele deles depois do dos e é ela elas ele eles em entre era eram éramos essa essas esse esses esta está estamos estão estar estas estava estavam estávamos este esteja estejam estejamos estes esteve estive estivemos estiver estivera estiveram estivéramos estiverem estivermos estivesse estivessem estivéssemos estou eu foi fomos for fora foram fôramos forem formos fosse fossem fôssemos fui há haja hajam hajamos hão havemos haver hei houve houvemos houver houvera houverá houveram houvéramos houverão houverei houverem houveremos houveria houveriam houveríamos houvermos houvesse houvessem houvéssemos isso isto já lhe lhes mais mas me mesmo meu meus minha minhas muito na não nas nem no nos nós nossa nossas nosso nossos num numa o os ou para pela pelas pelo pelos por qual quando que quem são se seja sejam sejamos sem ser será serão serei seremos seria seriam seríamos seu seus só somos sou sua suas também t

**Stopwords** são palavras que podem ser desconsideradas na análise. Por hora estamos usando uma lista padrão de stopwords para a lingua portuguesa, mas existe a possibilidade de incluirmos outras palavras que queremos eliminar no contexto desse sistema de recomendação, tendo assim uma lista de palavras mais customizadas.

In [7]:
teste_tags('title', 3)

Texto: Caso Bruno e Dom: 3º suspeito tem prisão temporária decretada pela Justiça do AM
Tags: decretada temporária 3º

Texto: Linguajar dos santarenos é diferenciado e cheio de identidade; 'égua, tu não vai
Tags: leso linguajar santarenos

Texto: Ex-premiê Shinzo Abe morre após ser baleado no Japão
Tags: shinzo abe premiê

Texto: Relator no STF, Fachin vota contra marco temporal para demarcação de terras indí
Tags: demarcação vota relator

Texto: 
Após 2 votos, pedido de vista suspende julgamento no STF sobre demarcação de te
Tags: demarcação terras julgamento



O resultado das tags para o titulo é mediano. Esse é um resultado esperado, uma vez que o modelo está rodando basicamente em configurações padrão e os textos dos titulos geralmente são curtos.

O resultado, em uma primeira análise, não parece ter uma qualidade muito alta, mas acredito que pode ser aproveitado.

In [8]:
teste_tags('caption', 12)

Texto: Jeferson da Silva Lima foi escoltado por agentes da Polícia Federal ao Fórum de 
Tags: escoltado jeferson custódia fórum audiência lima agentes silva município justiça federal polícia

Texto: As expressões santarenas não significam apenas o “sim” ou “não”, inclusive às ve
Tags: significam expressões santarenas pérola tapajós palavras coisas sim usadas inclusive vezes apenas

Texto: Ex-primeiro-ministro foi atingido por tiros de espingarda caseira quando discurs
Tags: discursava rígido nara raros caseira similares chocou espingarda japão armas controle atingido

Texto: Ministro defendeu que posse indígena é diferente da posse civil porque Constitui
Tags: demarcações parâmetro originário definirá posse defendeu constituição diferente garante terras indígena usado

Texto: Pelo marco temporal, índios só podem reivindicar demarcação de terras já ocupada
Tags: reivindicar demarcação 1988 ocupadas votou índios fachin edson constituição temporal marco relator



O caption é um pouco mais detalhado e informativo, esse texto permite a extração de mais tags e provavelmente elas serão de ais qualidade. Ainda percebo ruídos nas tags, mas acredito que o resultado pode ser usado para encontrar uma correlação entre as matérias.

In [9]:
teste_tags('body')

Texto: Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábad
Tags: phillips jeferson dom bruno pereira pelado indigenista dinha temporária amazonas prisão javari mortais amazônia restos embarcação oseney justiça jornalista decretou confirma lima custódia aras amarildo

Texto: Vista aérea de Santarém
Ádrio Denner/ AD Produções
O paraense tem um vocabulário
Tags: égua santarém santarenos expressões santareno indivíduo olha expressa figarella expressão intimidade denner jana descrença pará alguém significados vocabulário algo modo negação falar dúvida conjunção significam

Texto: Novo vídeo mostra que assassino de Shinzo Abe atirou 2 vezes pelas costas.
O ex-
Tags: abe shinzo japão premiê japonês ex ministro kyodo abenomics primeiro kishida nhk morte baleado poder reuters mesclava exterior país detido imperdoável cargo fumio atentado reformas

Texto: Relator no STF, Fachin vota contra marco temporal para demarcação de terras indí
Tags: indígenas temporal demarca

O corpo da matéria, por ser mais informativo e conter uma quantidade maior de texto,é esperado que tenhamos uma quantidade maior de tags relevantes.

O TfidfVectorizer está trazendo resultados interessantes, mas o chat gpt está sugerindo o keybert como uma opção com melhores resultados em testes de benchmark. Sendo assim, vou testar essa segunda biblioteca.

In [10]:
from keybert import KeyBERT

kw_model = KeyBERT(model='all-MiniLM-L6-v2')

for texto in df_itens['body'][:5]:
    keywords = kw_model.extract_keywords(texto, keyphrase_ngram_range=(1,1), stop_words=pt_stopwords, top_n=25)
    print(f"Texto: {texto[:80]}")
    print(f"Tags: {' '.join([ kw[0] for kw in keywords ])}\n")

c:\projects\fiap-1mlet\Projeto_5_Datathon\.venv\Lib\site-packages\urllib3\contrib\socks.py:50: DependencyWarning: SOCKS support in urllib3 requires the installation of optional dependencies: specifically, PySocks.  For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#socks-proxies
  warnings.warn(
c:\projects\fiap-1mlet\Projeto_5_Datathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Texto: Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábad
Tags: pelado prisão dinha mandado escoltado ocultação munição atividade prorrogado correção identificação peritos participação detidos segundo considerado lima cabeça embarcação momento relação feira foragido pede parecido

Texto: Vista aérea de Santarém
Ádrio Denner/ AD Produções
O paraense tem um vocabulário
Tags: diferenciado negação felicidade estudos informação estado identidade carinhoso linguístico realidade funcionam satisfação função santareno junção curiosidades santarenos descontentamento entonação significa característico conjunção devido égua produções

Texto: Novo vídeo mostra que assassino de Shinzo Abe atirou 2 vezes pelas costas.
O ex-
Tags: assassino significa shinzo estação realmente necessidade segundo desejo mencionava informação fabricação coração desempregado identificado atentado momento oito eleição popularidade ano legado caseira enforcamento atacado nacionalismo

Texto: Re

Em minha impressão inicial, o resultado do keybert não é muito melhor que o do TfidfVectorizer (se não for pior). Quero fazer uns testes com clusterização para comparar os resultados. 

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import IncrementalPCA

import nltk
from nltk.corpus import stopwords


def get_stopwords(language:str):
    nltk.download('stopwords', quiet=True)
    return list(stopwords.words(language))


def test_clusters(df, feature:str, n_clusters:int=10, n_components:int=10, chunk_size:int=10):
    df = df.copy()

    pt_stopwords = get_stopwords('portuguese')
    vectorizer = TfidfVectorizer(stop_words=pt_stopwords)
    pca = IncrementalPCA(n_components=n_components)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=64)

    X = vectorizer.fit_transform(df[feature])
    for chunk in range(0, X.shape[0], chunk_size):
        pca.partial_fit(X[chunk:chunk+chunk_size].toarray())
        kmeans.partial_fit(pca.transform(X[chunk:chunk+chunk_size].toarray()))
    
    df['cluster'] = kmeans.predict(df[feature])
    
    score = silhouette_score(X, df['cluster'])
    print(f"Silhouette Score: {score}")

    return df

In [41]:
df_itens_clustered = test_clusters(df_itens, 'caption', 25, 1000, 1000)

MemoryError: Unable to allocate 711. MiB for an array with shape (1000, 93236) and data type float64

In [24]:
df_itens_clustered = test_clusters(df_itens, 'caption', 150)

Silhouette Score: 0.01026533855804245


In [25]:
df_itens_clustered = test_clusters(df_itens, 'caption', 500)

Silhouette Score: 0.0127077048271191


In [27]:
df_itens_clustered = test_clusters(df_itens, 'caption', 5000)

MemoryError: Unable to allocate 3.47 GiB for an array with shape (5000, 93236) and data type float64